# 1. Authenticate & Import

In [1]:
import ee
# 网不好的时候自动重试ee.Initialize()
print("Try init ee...", end=" ")
eeInited, tryCounts = False, 0
while not eeInited:
    print("%d" % tryCounts, end=" ")
    try:
        ee.Initialize()
    except Exception as e:
        if (str(e)[0:23]=="Please authorize access"):
            ee.Authenticate()
        tryCounts += 1
    else:
        eeInited = True
        print("\nee initialized!")

import geemap


Try init ee... 0 1 2 3 4 5 6 7 8 
ee initialized!


In [ ]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import cv2

import gdal
import utils
import common
import dataset
import GetMapTiles
from models.ResNet import ResNet101

# 2. Import Earth Engine Data

In [ ]:
# Boundary and Grid-----------------------------------------------------------------------------------------------------
worldBoundary = ee.FeatureCollection("users/liuph/shape/WorldBoundary")
ChinaBoundary = ee.FeatureCollection("users/410093033/China")
WorldGrid5d = ee.FeatureCollection("users/liuph/shape/WorldGrid5dC5")

# Landcover products----------------------------------------------------------------------------------------------------

gong = ee.ImageCollection('users/wangyue/Gong2017Glc30') # gong's data
forest_gong = ee.ImageCollection(gong).qualityMosaic("b1").expression("b(0)==2?1:0").rename("Forestgong")

dataset = ee.ImageCollection('users/sunly3456/Forest2018ImageCollection') # ygs's data
forest_ygs =  ee.ImageCollection(dataset).qualityMosaic("b1").expression("b(0)==1?1:0").rename("Forestygs")

liu = ee.ImageCollection('users/wangyue/Glc2020Fcs30').select('b1') # liu's data
forest_liu = ee.ImageCollection(liu).qualityMosaic("b1").expression("b(0)>=50 && b(0)<=90?1:0").rename("Forestliu")

lc = ee.ImageCollection('users/sunly3456/GLC2020') # Chen's data
forest_chen = ee.ImageCollection(lc).qualityMosaic("b1").expression("b(0)==20?1:0").rename("Forestchen")

dataset = ee.Image('UMD/hansen/global_forest_change_2019_v1_7') # Hansen's data
start2000 = ee.Image(dataset).select('treecover2000').expression("b(0)>30&&b(0)<=100?1:0").rename("start00")
loss00_19 = ee.Image(dataset).expression("b(3)>1&&b(3)<=19?1:0").rename("loss00_19")
gain00_12 = ee.Image(dataset).expression("b(2)==1?1:0").rename("gain00_12")
forest_hs = start2000.add(gain00_12).subtract(loss00_19).expression("b(0)>0?1:0").rename("Foresths")

# Fusion of landcover products------------------------------------------------------------------------------------------
# forest_fuse = forest_gong.add(forest_ygs).add(forest_liu).add(forest_chen).add(forest_hs).rename("ForestFuse")
# forest_fuse = ee.ImageCollection('users/sysushiqian/forestFuse2020').min().rename("forest_fuse")
forest_fuse = ee.ImageCollection('users/sysushiqian/forestFuse2020_new').min().rename("ForestFuse")
forest23 = forest_fuse.expression('b(0)>=2 && b(0)<=3')

# Landsat8, cloud mask, median; add NDVI,NDWI,slop ---------------------------------------------------------------------
Landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
selVar = ee.List(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa'])
LC = Landsat.filter(ee.Filter.calendarRange(2020, 2020, 'year')).select(selVar).map(utils.maskL8sr)
selVar = ee.List(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'])
LC = LC.select(selVar).median()

ndvi = LC.normalizedDifference(['B5', 'B4']).rename('NDVI')
ndwi = LC.normalizedDifference(['B3', 'B5']).rename('NDWI')
DEM = ee.Image("MERIT/DEM/v1_0_3")
terrain = ee.Algorithms.Terrain(DEM)
slope = terrain.select('slope')
stratified_classes = ndvi.expression('(b(0)/0.2)').int().rename('STRATIFIED_CLASSES')

# Composite Image ------------------------------------------------------------------------------------------------------
LC_STN = LC.addBands(ndvi).addBands(ndwi).addBands(slope).addBands(stratified_classes).float()
selVar1 = ee.List(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','slope'])

# Validation points ----------------------------------------------------------------------------------------------------
valPoints_all = ee.FeatureCollection("users/410093033/wuzhengyi3").merge(ee.FeatureCollection("users/410093033/youzhengyi1"))

# 3. Prepare CNN model & data path

In [ ]:

ckptPath = r"./CKPT/ResNet101_GE17_1206/ckpt.pth"


model = ResNet101(in_ch=3, n_classes=5)
model.to(torch.device('cuda:0'))
assert os.path.exists(ckptPath), "ckpt dosen't exists"
ckpt = torch.load(ckptPath, map_location=torch.device('cuda:0'))
model.load_state_dict(ckpt["model"])
# model = nn.DataParallel(model, device_ids=(0, ))

# 4. Operate on selected 5d grid

In [ ]:
ID5d = 1038

dataPath = r"E:\SZT\Data\ff0105_ge01"
exportAssetPath = "users/thomasshen99/ForestPred_onlySure_1212"
exportNamePrefix = exportAssetPath.split('/')[-1]

time_start = time.time()
dataPath_curGrid = dataPath + "./grid%d" % ID5d
selected5d = WorldGrid5d.filterMetadata("ID", "equals", ID5d).first()
llLng, llLat = selected5d.getNumber("llLng").getInfo(), selected5d.getNumber("llLat").getInfo()
grid05d, grid01d = utils.get0501Grid(selected5d, forest23)

IDlist_01d = grid01d.reduceColumns(ee.Reducer.toList(), ["ID_01d"]).get("list").getInfo()

In [ ]:
# all_unsure_CNNpred, all_unsure_CNNpred_offseted = [], [] # 需要导出样本点为csv时使用
for tileNum in range(10):
    # 1. 生成随机森林样本-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    idList_05d = list(range(tileNum*10, (tileNum+1)*10, 1))
    idList_01d = IDlist_01d[tileNum*10:(tileNum+1)*10]

    print("\n[Grid%d:Tile%d] %s" % (ID5d, tileNum, time.ctime()))
    print("%s" % "-"*80)
    while(not common.checkExists_ffge(dataPath, ID5d, idList_05d, idList_01d)):
        time.sleep(120)

    print("Generating samples for RF: ")
    llListFS_list, llListFU_list, llListFU_offseted_list = [], [], []
    for idx, ID05d in enumerate(idList_05d):
        ID01d = grid01d.filterMetadata("ID_05d", "equals", ID05d).first().getNumber("ID_01d").getInfo()
        # get full llList
        llList05_full = common.getLonLatListFromImage(dataPath_curGrid + "/ff05_%d_%d.tif" % (ID5d, ID05d))
        llList01_full = common.getLonLatListFromImage(dataPath_curGrid + "/ff01_%d_%d_%d.tif" % (ID5d, ID05d, ID01d))
        llList05_sure_pos, llList05_sure_neg, llList05_unsure = common.getSP_SN_USsplit(llList05_full)
        llList01_sure_pos, llList01_sure_neg, llList01_unsure = common.getSP_SN_USsplit(llList01_full)
        llList_sure_pos, llList_sure_neg, llList_unsure = llList05_sure_pos, llList05_sure_neg, llList01_unsure

        # sample from llList
        sampleNum_sure = min([2500, llList_sure_pos.shape[0], llList_sure_neg.shape[0]])
        sampleNum_unsure = min(3200, llList_unsure.shape[0])
        np.random.seed(0)

        sp_idx = np.random.choice(np.arange(llList_sure_pos.shape[0]), size=sampleNum_sure, replace=False)
        sn_idx = np.random.choice(np.arange(llList_sure_neg.shape[0]), size=sampleNum_sure, replace=False)
        us_idx = np.random.choice(np.arange(llList_unsure.shape[0]), size=sampleNum_unsure, replace=False)
        llList_sure_pos, llList_sure_neg, llList_unsure = llList_sure_pos[sp_idx, ...], llList_sure_neg[sn_idx, ...], llList_unsure[us_idx, ...]

        # predict unsure area label with CNN
        gePath_cur = dataPath_curGrid + "/ge_%d_%d_%d.tif" % (ID5d, ID05d, ID01d)
        assert os.path.exists(gePath_cur), "google earth image not found"  
        llList_forest_sure, llList_forest_unsure, llList_forest_unsure_offseted = common.getRFSampleList(llList_sure_pos,llList_sure_neg, llList_unsure, model, gePath=gePath_cur, desc="tile%d ID05d=%d:" % (tileNum, ID05d))

        # append
        llListFS_list.append(llList_forest_sure)
        llListFU_list.append(llList_forest_unsure)
        llListFU_offseted_list.append(llList_forest_unsure_offseted)

# 需要导出样本点为csv时使用
        # all_unsure_CNNpred_offseted.extend(llList_forest_unsure_offseted)
        # all_unsure_CNNpred.extend(llList_forest_unsure)
# np.savetxt('./grid1038_unsure.csv', all_unsure_CNNpred, fmt="%s", delimiter=",")
# np.savetxt('./grid1038_unsure_CAMoffseted.csv', all_unsure_CNNpred_offseted, fmt="%s", delimiter=",")

    # 2. 训练随机森林，并预测当前Tile---------------------------------------------------------------------------------------------------------------------------------------------------------------
    print("RF Training & predicting...")
    predForest_list = []
    for idx, ID05d in enumerate(idList_05d):
        # ****在此设置使用的RF样本****        
        sample_train = common.getRFSampleFC(LC_STN, llListFS_list[idx] + llListFU_offseted_list[idx])
        # sample_train = common.getRFSampleFC(LC_STN, llListFS_list[idx] + llListFU_list[idx])

        classifier = ee.Classifier.smileRandomForest(numberOfTrees=200, variablesPerSplit=9, minLeafPopulation=1, bagFraction=0.5, maxNodes=None, seed=0).train(sample_train, "forest", selVar1)
        cur_geom05d = grid05d.filterMetadata("ID_05d", "equals", ID05d).first().geometry()
        predForest = LC_STN.clip(cur_geom05d).select(selVar1).classify(classifier)
        predForest_list.append(predForest)
    predTile = ee.ImageCollection(predForest_list).mosaic().uint8().rename("forest")

    # 3. 导出-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    exportRegion = ee.Algorithms.GeometryConstructors.Rectangle([llLng+(idList_05d[0]//10)*0.5, llLat+(idList_05d[0]%10)*0.5, llLng+(idList_05d[-1]//10+1)*0.5, llLat+(idList_05d[-1]%10+1)*0.5])    
    exportName = "%s_grid%d_tile%d" % (exportNamePrefix, ID5d, tileNum)

    try:
        ee.data.listAssets({"parent":"projects/earthengine-legacy/assets/%s" % (exportAssetPath)})
    except Exception as e:
        if str(e)[-10:]=="not found.":
            ee.data.createAsset({"type":"Folder"}, opt_path="projects/earthengine-legacy/assets/%s" % (exportAssetPath))
    # geemap.ee_export_image_to_drive(predTile.clip(exportRegion), exportName, folder="GEEexport", region=exportRegion, scale=30, max_pixels=1.0E13, file_format='GeoTIFF')
    task = ee.batch.Export.image.toAsset(predTile.clip(exportRegion), description=exportName, assetId="projects/earthengine-legacy/assets/%s/%s" % (exportAssetPath, exportName), pyramidingPolicy={"forest":"mode"}, region=exportRegion, scale=30, maxPixels=1e13)
    task.start()
    print("Predicting finished. Grid%d tile%d exporting task started...\n" % (ID5d, tileNum))

# 6. Metrics

In [ ]:
productList = [forest_gong, forest_ygs, forest_liu, forest_chen, forest_hs]
productNameList = ["Forestgong", "Forestygs", "Forestliu", "Forestchen", "Foresths"]
predDict = {"forestPred_onlySure":"users/thomasshen99/ForestPred_onlySure_1212", 
            "forestPred_withUnsureByCNN":"users/thomasshen99/ForestPred_original_1212", 
            "forestPred_withUnsureByCNN_CAMoffseted":"users/thomasshen99/ForestPred_1211"}

valPoints_cur = utils.getBalancedValPoints(valPoints_all, region=selected5d.geometry(), maxRate=4)

# predictions
for key in predDict.keys():
    pred = ee.ImageCollection(list(map(lambda x:x['id'], ee.data.listAssets({"parent":"projects/earthengine-legacy/assets/%s" % predDict[key]})['assets']))).mosaic().rename(key)
    productList.append(pred)
    productNameList.append(key)

# stack all product in productList & sample value to valPoints_cur
productComp = ee.Image.cat(productList)
valPoints_cur_sampled = productComp.sampleRegions(collection=valPoints_cur, geometries=False, scale=30)

# calculate metrics and forest area, combine all in to a ee.FeatureCollection
areaDict = ee.Dictionary(utils.CalcArea(productComp, selected5d.geometry()))
metricsDictList = ee.List(productNameList).map(utils.getCalcMetricsFunc(valPoints_cur_sampled, areaDict))
metricsFC = ee.FeatureCollection(metricsDictList.map(lambda x:ee.Feature(None, x)))

# geemap.ee_export_vector(metricsFC, "./foo.csv")

In [10]:
mDL = metricsDictList.getInfo()

# 5. View GEE data with geemap

In [ ]:
Map = geemap.Map(center=[56, 26], zoom=12, add_google_map=True)
Map.add_basemap('Google Satellite')

Map.addLayer(LC_STN.clip(selected5d), vis_params={"max":2000, "min":0, "bands":['B4', 'B3', 'B2']}, name="LC08")

Map.addLayer(forest_gong.clip(selected5d), vis_params={"max":1, "min":0, "palette":["FF0000", "00FF00"]}, name="forest_gong")
Map.addLayer(forest_fuse.clip(selected5d), vis_params={"max":5, "min":0, "palette":["FF0000", "FF0000", "FFFF00", "FFFF00", "00FF00", "00FF00"]}, name="forest_fuse", opacity=1)


Map.addLayer(productComp, vis_params={"max":1, "min":0, "palette":["FF0000", "00FF00"]}, name="forest_pred", opacity=1)


Map.addLayerControl()
Map